In [4]:
import logging
import multiprocessing
from pprint import pprint
import pyedflib
from pyedflib import highlevel

import datetime
import matplotlib.pyplot as plt

from config.paths import PATHS, Dataset
from config.paths import PatientDir

In [7]:
ptnt_dir = PatientDir(PATHS.for_mayo_dir / 'B52K3P3G')
# ptnt_dir = PatientDir(PATHS.competition_dir / 'P1')

edf_path = next(ptnt_dir.edf_dir.iterdir())  # get the first edf
edf_path

PosixPath('/data/home/webb/UNEEG_data/20240201_UNEEG_ForMayo/B52K3P3G/edf_data/B52K3P3G_2021-10-24_21-56-04.edf')

In [8]:
# Simply read a timezone
header = highlevel.read_edf_header(str(edf_path))
local_time: str = header['annotations'][0][2]
local_time = local_time.removeprefix('LOCAL TIME = ')
print(local_time)

UTC+02h


# Find all time zones per patient

In [9]:
def find_ptnt_timezones(ptnt_dir: PatientDir) -> dict:
    logging.info(f"Processing {ptnt_dir.name}")
    timezones = set()
    problematic_files = []
    for edf_path in ptnt_dir.edf_dir.iterdir():
        logging.debug(f"Processing {edf_path.name}")
        try:
            header = highlevel.read_edf_header(str(edf_path))
            local_time: str = header['annotations'][0][2]
            local_time = local_time.removeprefix('LOCAL TIME = ')
            timezones.add(local_time)
        except:
            problematic_files.append(edf_path)
    timezones = list(timezones)
    timezones.sort()
    logging.info(f"Finished {ptnt_dir.name}")
    return {'timezones': timezones, 'problematic files': problematic_files}

In [10]:
logging.basicConfig(level=logging.INFO, format='[%(levelname)s]: %(message)s')

# for pd in PATHS.patient_dirs(include_invalid_ptnts=True):
#     print('=== Processing', pd.name)
#     ptnt_timezones[pd.name] = find_ptnt_timezones(pd)

ptnt_dirs = PATHS.patient_dirs([Dataset.for_mayo, Dataset.uneeg_extended], include_invalid_ptnts=True)
with multiprocessing.Pool() as pool:
    ptnt_timezones = pool.map(find_ptnt_timezones, ptnt_dirs)
ptnt_timezones = {pd.name: tz for pd, tz in zip(ptnt_dirs, ptnt_timezones)}

[INFO]: Processing K37N36L4D
[INFO]: Processing G39B4L9E
[INFO]: Processing M39K4B3C
[INFO]: Processing B52K3P3G
[INFO]: Processing P4Hk23M7L
[INFO]: Processing P73M2F6H
[INFO]: Processing D63Q51K2N
[INFO]: Processing E85L95P2H
[INFO]: Processing A4RW34Z5B
[INFO]: Processing F5TW95P3X
[INFO]: Processing K53T36N7F
[INFO]: Processing E15T65H3Z
[INFO]: Processing L3GS57K2T
[INFO]: Finished K53T36N7F
[INFO]: Finished F5TW95P3X
[INFO]: Finished M39K4B3C
[INFO]: Finished P73M2F6H
[INFO]: Finished D63Q51K2N
[INFO]: Finished E85L95P2H
[INFO]: Finished E15T65H3Z
[INFO]: Finished B52K3P3G
[INFO]: Finished P4Hk23M7L
[INFO]: Finished A4RW34Z5B
[INFO]: Finished L3GS57K2T
[INFO]: Finished K37N36L4D
[INFO]: Finished G39B4L9E


In [11]:
ptnt_timezones

{'K37N36L4D': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'G39B4L9E': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'M39K4B3C': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'B52K3P3G': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'F5TW95P3X': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'D63Q51K2N': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'P4Hk23M7L': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'P73M2F6H': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'E85L95P2H': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'A4RW34Z5B': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'L3GS57K2T': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'K53T36N7F': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic files': []},
 'E15T65H3Z': {'timezones': ['UTC+01h', 'UTC+02h'], 'problematic fil